<h1>Data</h1>

In [7]:
%matplotlib inline
from mysql import connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

mysql_connector = connector.connect(host="127.0.0.1", database="aq", 
                                    user="root", password="!QAZ2wsx#EDC")
data = pd.read_sql("""SELECT close, volume FROM future_trade
                      WHERE code='%s' AND type='%s' """ % ("RB", "5"), con=mysql_connector)
mysql_connector.close()
data["chg"] = data.close.diff()
data.iloc[0, 2] = 0

lag_rtn = 10
lag_vol = 10
y = data.chg[max(lag_rtn, lag_vol):]
y.index = range(len(y))
y = y.apply(lambda x: x > 0)
X = pd.DataFrame(columns=range(lag_rtn + lag_vol))
for idx, row in data.iterrows():
    if idx >= lag_rtn and idx >= lag_vol:
        vol = data.iloc[(idx - lag_vol):idx, 1]
        rtn = data.iloc[(idx - lag_rtn):idx, 2]
        xrow = pd.concat([rtn, vol], ignore_index=True)
        X = X.append(xrow, ignore_index=True)

print("Std:", data.chg.std())
print("Mean:", data.chg.mean())

Std: 6.766430139868762
Mean: -0.005596138237430048


# Logistic Regression

In [4]:
from sklearn.linear_model import LogisticRegression
k_fold = 50
stride = round(len(data.chg)/k_fold)
for i in range(0, k_fold):
    start = i*stride
    stop = i*stride+stride-1
    if (i == k_fold-1):
        stop = len(data.chg)
    X_test = X[start:stop]
    y_test = y[start:stop]
    X_train = pd.concat([X[0:start-1], X[stop:]])
    y_train = pd.concat([y[0:start-1], y[stop:]])    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    print("i=%s, %.2f%s" % (i, np.sum(model.predict(X_test) == y_test)*100/stride, "%"))

i=0, 64.22%
i=1, 64.39%
i=2, 64.28%
i=3, 62.83%
i=4, 61.81%
i=5, 59.67%
i=6, 58.82%
i=7, 58.30%
i=8, 56.78%
i=9, 56.19%


# Navie Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB
k_fold = 50
stride = round(len(data.chg)/k_fold)
for i in range(0, k_fold):
    start = i*stride
    stop = i*stride+stride-1
    if (i == k_fold-1):
        stop = len(data.chg)
    X_test = X[start:stop]
    y_test = y[start:stop]
    X_train = pd.concat([X[0:start-1], X[stop:]])
    y_train = pd.concat([y[0:start-1], y[stop:]])    
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("i=%s, %.2f%s" % (i, np.sum(model.predict(X_test) == y_test)*100/stride, "%"))

i=0, 61.81%
i=1, 60.76%
i=2, 64.29%
i=3, 65.05%
i=4, 68.19%
i=5, 62.10%
i=6, 67.24%
i=7, 62.29%
i=8, 60.95%
i=9, 62.19%
i=10, 57.14%
i=11, 62.10%
i=12, 62.67%
i=13, 62.57%
i=14, 67.71%
i=15, 63.71%
i=16, 64.38%
i=17, 59.81%
i=18, 63.71%
i=19, 65.62%
i=20, 62.67%
i=21, 64.95%
i=22, 60.95%
i=23, 58.29%
i=24, 58.38%
i=25, 56.86%
i=26, 59.52%
i=27, 55.90%
i=28, 57.14%
i=29, 55.90%
i=30, 56.00%
i=31, 54.29%
i=32, 58.67%
i=33, 53.81%
i=34, 51.71%
i=35, 50.76%
i=36, 54.67%
i=37, 53.90%
i=38, 53.14%
i=39, 53.52%
i=40, 55.43%
i=41, 54.86%
i=42, 53.71%
i=43, 55.81%
i=44, 53.24%
i=45, 52.00%
i=46, 54.00%
i=47, 53.33%
i=48, 55.52%
i=49, 53.33%
